In [1]:
import pandas as pd
import numpy as np
import xlrd
import sklearn
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import time
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import math
import datetime
import scipy.stats
import itertools
import pyodbc 
import oracledb 
import openpyxl
import cx_Oracle

# Data extraction from database

# from Oracle SQL

In [2]:
aircraft_template_from_confluence = pd.read_csv('aircraft_template.csv')
aircraft_template_from_confluence

,Engine Variant,Engine S/N,Engine Model ID,Engine Model,Aircraft Model,Aircraft Model Variant,Aircraft Model ID Code,EHM Remarks,MTX Remarks
0,PE331-10R-502C,NaN,NaN,NaN,AIRBUS C-212-CD (EHAI),NaN,EHCD,NaN,Added 27 Oct 2023
1,TPE331-10R-512C,NaN,NaN,NaN,AIRBUS C-212-CD (EHAI),NaN,EHCD,NaN,Added 27 Oct 2023
2,TPE3-10R-513C,NaN,NaN,NaN,AIRBUS C-212-CD (EHAI),NaN,EHCD,NaN,Added 27 Oct 2023
3,AS907-1-1A,NaN,NaN,NaN,CH 300,NaN,C300,NaN,NaN
4,AS907-2-1A,NaN,NaN,NaN,CH350,NaN,C350,NaN,NaN
...,...,...,...,...,...,...,...,...,...
435,TPE331-12UHR-701G,NaN,NaN,TPE331-12JR,CASA 212-400 with -12engs,NaN,NO AIRCRAFT TEMPLATE,NaN,NaN
436,TPE331-14GR/14HR,NaN,EJ41 - CAMP MAINTAINED,TPE331-14GR-801Z,Marsh S2T / CDF,NaN,NO AIRCRAFT TEMPLATE,NaN,NaN
437,TPE331-5,NaN,NaN,TPE331-5-25xM,Mitsubishi MU-2B-26A / P-Model,NaN,NaN,NaN,NaN
438,TPE331-6/-10,NaN,E331 - CAMP MAINTAINED,TPE331-6/6A-251M,Mitsubishi MU-2B / J/K/L/M-Models,NaN,MU2B-60 – MU02 -,NaN,NaN


In [3]:
aircraft_template_from_confluence [aircraft_template_from_confluence['Aircraft Model']=='PA-46-500TP' ]

,Engine Variant,Engine S/N,Engine Model ID,Engine Model,Aircraft Model,Aircraft Model Variant,Aircraft Model ID Code,EHM Remarks,MTX Remarks
87,PT6A-34,NaN,NaN,NaN,PA-46-500TP,NaN,PA47,NaN,NaN
105,PT6A-42A,NaN,NaN,NaN,PA-46-500TP,NaN,PA47,NaN,NaN


In [4]:
 aircraft_template_from_confluence['MTX Remarks'] .unique()

array(['Added\xa027 Oct 2023\xa0', nan,
       'Incorrect Combination - should be E280 engine template (N696HC--Trend Support only)',
       '\xa0\xa0Correct Combination - E280 engine template',
       'Corrected 30 Oct 2023\xa0', 'added 30 Oct 2023\xa0',
       '\xa0Correct Combination', 'Incorrect Combination,',
       'I changed the Engine Variant\xa0',
       'to the correct JT15D-4\xa0per MTR entry.\xa0',
       'D. Martin 10-23-2017.', '\xa0\xa0Correct Combination',
       '\xa0Correct Combination\xa0',
       'If FAST ID is not provided in CAF, put a note in comment to get FAST ID from customer.',
       'Customer only (AWS Air). Do not use for any other customer.',
       '\xa0To be verified', '\xa0Incorrect Combination',
       '\xa0Correct Combination - E056 engine template',
       '\xa0\xa0Incorrect Combination', '\xa0\xa0To be verified',
       '\xa0Correct Combination - EC24 engine template', '\xa0 ',
       '\xa0Correct Combination - E024 engine template',
       '\xa0Co

In [5]:
g = aircraft_template_from_confluence [['Aircraft Model','Engine Variant']]
g

,Aircraft Model,Engine Variant
0,AIRBUS C-212-CD (EHAI),PE331-10R-502C
1,AIRBUS C-212-CD (EHAI),TPE331-10R-512C
2,AIRBUS C-212-CD (EHAI),TPE3-10R-513C
3,CH 300,AS907-1-1A
4,CH350,AS907-2-1A
...,...,...
435,CASA 212-400 with -12engs,TPE331-12UHR-701G
436,Marsh S2T / CDF,TPE331-14GR/14HR
437,Mitsubishi MU-2B-26A / P-Model,TPE331-5
438,Mitsubishi MU-2B / J/K/L/M-Models,TPE331-6/-10


In [6]:
# Grouping by 'Aircraft Model' and 'Engine Variant' and calculating the count
grouped_df = g.groupby(['Aircraft Model', 'Engine Variant']).size().reset_index(name='Count')
grouped_df

,Aircraft Model,Engine Variant,Count
0,1124 WESTWIND,TFE731-3-1E,1
1,1124 WESTWIND,TFE731-3-1G,1
2,1124 WESTWIND,TFE731-3D-1G,1
3,1124 WESTWIND,TFE731-3R-1G,1
4,1124A WESTWIND,TFE731-3-1G,1
...,...,...,...
354,TBM 930,PT6A-66D2,1
355,TBM 930,PT6A-66D3,1
356,TEXAN T-6C,PT6A-68,1
357,TWIN OTTER (TC),PT6A-27,1


In [7]:
grouped_df['Count'].unique()

array([1, 3, 2, 8, 4], dtype=int64)

In [8]:
grouped_df [ grouped_df['Count']!=1 ]

,Aircraft Model,Engine Variant,Count
43,ATR 42-600,PW127M,3
44,ATR42-300,PW120,3
45,ATR42-300,PW121,3
46,ATR42-600,PW121A,2
47,ATR42-600,PW127E,2
48,ATR72,PW124B,2
49,ATR72,PW127,8
50,ATR72,PW127F,2
51,ATR72,PW127M,4
58,BAE Jetstream 3100,NO ENGINE TEMPLATE,2


In [9]:
g[g['Aircraft Model']=='DASH 8 Q200']

,Aircraft Model,Engine Variant
221,DASH 8 Q200,PW123
222,DASH 8 Q200,PW123
227,DASH 8 Q200,PW123B
228,DASH 8 Q200,PW123B
232,DASH 8 Q200,PW123C
233,DASH 8 Q200,PW123C
234,DASH 8 Q200,PW123C
235,DASH 8 Q200,PW123D
236,DASH 8 Q200,PW123D
237,DASH 8 Q200,PW123E


In [10]:
data = pd.read_csv('powerBI_data.csv')
data

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,Engine Model,Engine Variant,Category
0,003025,INACTIVE,PW119C,PW119B,NaN,NaN,DORNIER 328-100,3281,NaN,NaN,Information not available in Confluence
1,003026,INACTIVE,PW119B,PW119C,NaN,NaN,DORNIER 328-100,3281,PW119B/C,PW119B,Matching with Documentation
2,003106,INACTIVE,PW119C,PW119B,NaN,NaN,DORNIER 328-100,3281,NaN,NaN,Information not available in Confluence
3,2-ANVL,ACTIVE,PW307D,PW307D,PW307D,NaN,FALCON 8X,FA8X,NaN,PW307D,Matching with Documentation
4,2-NYAW,INACTIVE,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,NaN,FALCON 50,3645,NaN,TFE731-3D-1C,Matching with Documentation
...,...,...,...,...,...,...,...,...,...,...,...
1152,ZS-LAC,INACTIVE,TFE731-40-1C,TFE731-40-1C,TFE731-40-1C,NaN,FALCON 50EX,3647,NaN,TFE731-40-1C,Matching with Documentation
1153,ZS-MGS,INACTIVE,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,NaN,FALCON 50,3645,NaN,TFE731-3D-1C,Matching with Documentation
1154,ZS-MGT,INACTIVE,TFE731-5BR-1C,TFE731-5BR-1C,TFE731-5BR-1C,NaN,FALCON 900B,6892,NaN,TFE731-5BR-1C,Matching with Documentation
1155,ZS-XCO,INACTIVE,PW127,PW127F,NaN,NaN,ATR72,AT72,PW124B/PW127/127E/F/M/N,PW127,Matching with Documentation


In [11]:
data [ (data['ACMODEL'] == 'DASH 8 Q200') and (data['ACMODEL'] == 'DASH 8 Q200') ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
w = data [ data['ACMODEL']=='DHC-6' ]
w

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,Engine Model,Engine Variant,Category
77,C-FATM,INACTIVE,PT6A-28,PT6A-27,NaN,NaN,DHC-6,VK01,NaN,NaN,Information not available in Confluence


In [25]:
t = pd.read_csv('acmodel_variants.csv')
t

,ACMODEL,Engine Variant 1,Engine Variant 2,Engine Variant 3
0,DORNIER 328-100,PW119B,NaN,NaN
1,FALCON 8X,PW307D,NaN,NaN
2,FALCON 50,TFE731-3-1C,NaN,NaN
3,EMB-120ER,PW118A,NaN,NaN
4,FALCON 7X,PW307A,NaN,NaN
...,...,...,...,...
67,ATR42-300,PW120,NaN,NaN
68,DHC-6-100/200/300,PT6A-20,PT6A-27,NaN
69,ATR 72-212A (600 SERIES),PW127M,NaN,NaN
70,BELL 412HP,PT6T-3DF,NaN,NaN


In [26]:
t [t['ACMODEL']=='DASH 8 Q200']

,ACMODEL,Engine Variant 1,Engine Variant 2,Engine Variant 3
46,DASH 8 Q200,PW123D,NaN,NaN


In [21]:
# Merge data with t on 'ACMODEL'
merged = pd.merge(data, t, on='ACMODEL', how='left')

# Assign the matching variant to 'verified_variant' column
merged['verified_variant'] = merged.apply(
    lambda row: row['Engine Variant'] if row['Engine Variant 1'] == row['Engine Variant'] else None, axis=1
)

# Drop the extra 'Engine Variant' column from the merge
merged.drop(columns=['Engine Variant'], inplace=True)


In [22]:
merged

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,Engine Model,Category,Engine Variant 1,Engine Variant 2,Engine Variant 3,verified_variant
0,003025,INACTIVE,PW119C,PW119B,NaN,NaN,DORNIER 328-100,3281,NaN,Information not available in Confluence,PW119B,NaN,NaN,None
1,003026,INACTIVE,PW119B,PW119C,NaN,NaN,DORNIER 328-100,3281,PW119B/C,Matching with Documentation,PW119B,NaN,NaN,PW119B
2,003106,INACTIVE,PW119C,PW119B,NaN,NaN,DORNIER 328-100,3281,NaN,Information not available in Confluence,PW119B,NaN,NaN,None
3,2-ANVL,ACTIVE,PW307D,PW307D,PW307D,NaN,FALCON 8X,FA8X,NaN,Matching with Documentation,PW307D,NaN,NaN,PW307D
4,2-NYAW,INACTIVE,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,NaN,FALCON 50,3645,NaN,Matching with Documentation,TFE731-3-1C,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,ZS-LAC,INACTIVE,TFE731-40-1C,TFE731-40-1C,TFE731-40-1C,NaN,FALCON 50EX,3647,NaN,Matching with Documentation,TFE731-40-1C,NaN,NaN,TFE731-40-1C
1153,ZS-MGS,INACTIVE,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,NaN,FALCON 50,3645,NaN,Matching with Documentation,TFE731-3-1C,NaN,NaN,None
1154,ZS-MGT,INACTIVE,TFE731-5BR-1C,TFE731-5BR-1C,TFE731-5BR-1C,NaN,FALCON 900B,6892,NaN,Matching with Documentation,TFE731-5BR-1C,NaN,NaN,TFE731-5BR-1C
1155,ZS-XCO,INACTIVE,PW127,PW127F,NaN,NaN,ATR72,AT72,PW124B/PW127/127E/F/M/N,Matching with Documentation,PW127F,PW127M,NaN,None


In [4]:
conn = oracledb.connect('bezyb/camp123@dbm04-scan1.campsys.com:1521/exadrdb.campsys.com')

In [5]:
engine_trim_mismatch = pd.read_sql(f""" 
    SELECT *
FROM (
    SELECT E.*, MM.ACMODELID, ACMODELID || '-' || ENGMODEL AS IDENTIFIER, 'TRIM-MISMATCHING' AS CAT
    FROM (
        SELECT 
            AIRCRAFT_ID,
            TREND.V_LINKED_ASSETS.REGNO,
            CASE WHEN AIRCRAFT_MTX_STATUS = 1 THEN 'INACTIVE' ELSE 'ACTIVE' END AS MTX_STATUS,
            vv.engmodel, vv.customer, vv.acmodel, vv.esn,
            MAX(DECODE(POSITION, 'NO. 1', ENGINE_MODEL_TRIM)) AS POS_1,
            MAX(DECODE(POSITION, 'NO. 2', ENGINE_MODEL_TRIM)) AS POS_2,
            MAX(DECODE(POSITION, 'NO. 3', ENGINE_MODEL_TRIM)) AS POS_3,
            MAX(DECODE(POSITION, 'NO. 4', ENGINE_MODEL_TRIM)) AS POS_4
        FROM TREND.V_LINKED_ASSETS
        JOIN trend.v_analyzer_ac_dashboard vv 
            ON vv.acassetid = AIRCRAFT_ID 
            AND engpos = 1
        WHERE TREND.V_LINKED_ASSETS.ECTM_SUBSCRIPTION = 'Y'
        GROUP BY AIRCRAFT_ID, TREND.V_LINKED_ASSETS.REGNO, AIRCRAFT_MTX_STATUS, vv.engmodel, vv.customer, vv.acmodel, vv.esn
        ORDER BY TREND.V_LINKED_ASSETS.REGNO
    ) E
    JOIN (
        SELECT DISTINCT 
            regno, 
            P.ASSET_ID, 
            CAMP_TAPE || CAMP_MODEL AS ACMODELID, 
            MODEL_NAME 
        FROM csi.profile P
        JOIN csi.model M 
            ON M.model_id = P.model_id
        WHERE ECTM_SUBSCRIPTION = 'Y' 
        AND M.PROFILETYPE_LOOKUP = 'AIRCRAFT'
    ) MM 
    ON MM.ASSET_ID = E.AIRCRAFT_ID
    WHERE POS_1 <> POS_2 OR POS_1 <> POS_3 OR POS_2 <> POS_3

    UNION ALL

    SELECT E.*, MM.ACMODELID, ACMODELID || '-' || ENGMODEL AS IDENTIFIER, 'TRIM-MATCHING' AS CAT
    FROM (
        SELECT 
            AIRCRAFT_ID,
            TREND.V_LINKED_ASSETS.REGNO,
            CASE WHEN AIRCRAFT_MTX_STATUS = 1 THEN 'INACTIVE' ELSE 'ACTIVE' END AS MTX_STATUS,
            vv.engmodel, vv.customer, vv.acmodel, vv.esn,
            MAX(DECODE(POSITION, 'NO. 1', ENGINE_MODEL_TRIM)) AS POS_1,
            MAX(DECODE(POSITION, 'NO. 2', ENGINE_MODEL_TRIM)) AS POS_2,
            MAX(DECODE(POSITION, 'NO. 3', ENGINE_MODEL_TRIM)) AS POS_3,
            MAX(DECODE(POSITION, 'NO. 4', ENGINE_MODEL_TRIM)) AS POS_4
        FROM TREND.V_LINKED_ASSETS
        JOIN trend.v_analyzer_ac_dashboard vv 
            ON vv.acassetid = AIRCRAFT_ID 
            AND engpos = 1
        WHERE TREND.V_LINKED_ASSETS.ECTM_SUBSCRIPTION = 'Y'
        GROUP BY AIRCRAFT_ID, TREND.V_LINKED_ASSETS.REGNO, AIRCRAFT_MTX_STATUS, vv.engmodel, vv.customer, vv.acmodel, vv.esn
        ORDER BY TREND.V_LINKED_ASSETS.REGNO
    ) E
    JOIN (
        SELECT DISTINCT 
            regno, 
            P.ASSET_ID, 
            CAMP_TAPE || CAMP_MODEL AS ACMODELID, 
            MODEL_NAME 
        FROM csi.profile P
        JOIN csi.model M 
            ON M.model_id = P.model_id
        WHERE ECTM_SUBSCRIPTION = 'Y' 
        AND M.PROFILETYPE_LOOKUP = 'AIRCRAFT'
    ) MM 
    ON MM.ASSET_ID = E.AIRCRAFT_ID
    WHERE POS_1 = POS_2 AND POS_1 = POS_3 AND POS_2 = POS_3
)

""", conn)

engine_trim_mismatch

C:\Users\bezyb\AppData\Local\Temp\ipykernel_34376\3099285848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  engine_trim_mismatch = pd.read_sql(f"""


,AIRCRAFT_ID,REGNO,MTX_STATUS,ENGMODEL,CUSTOMER,ACMODEL,ESN,POS_1,POS_2,POS_3,POS_4,ACMODELID,IDENTIFIER,CAT
0,70903,003025,INACTIVE,PW119C,SIERRA NEVADA CORPORATION DO328,DORNIER 328-100,AB0033,PW119C,PW119B,None,None,3281,3281-PW119C,TRIM-MISMATCHING
1,70904,003026,INACTIVE,PW119B,SIERRA NEVADA CORPORATION DO328,DORNIER 328-100,116106,PW119B,PW119C,None,None,3281,3281-PW119B,TRIM-MISMATCHING
2,71905,003106,INACTIVE,PW119C,SIERRA NEVADA CORPORATION DO328,DORNIER 328-100,116161,PW119C,PW119B,None,None,3281,3281-PW119C,TRIM-MISMATCHING
3,131667,5R-EJD,INACTIVE,PW127F,HANGAR - SAF,ATR72,ED0062,PW127F,PW127M,None,None,AT72,AT72-PW127F,TRIM-MISMATCHING
4,65741,5Y-BZI,INACTIVE,PW120A,ALS LIMITED,DASH 8,120304,PW120A,PW121,None,None,6190,6190-PW120A,TRIM-MISMATCHING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,1016,ZS-DEX,ACTIVE,TFE731-60-1C,CAMOSA,900EX,112332,TFE731-60-1C,TFE731-60-1C,TFE731-60-1C,None,6893,6893-TFE731-60-1C,TRIM-MATCHING
1138,19213,ZS-DFJ,ACTIVE,TFE731-5BR-1C,ZENITH AIR CHARTER,FALCON 900,101226,TFE731-5BR-1C,TFE731-5BR-1C,TFE731-5BR-1C,None,6892,6892-TFE731-5BR-1C,TRIM-MATCHING
1139,862,ZS-LAC,INACTIVE,TFE731-40-1C,LAC AVIATION (PTY) LTD,FALCON 50EX,115116,TFE731-40-1C,TFE731-40-1C,TFE731-40-1C,None,3647,3647-TFE731-40-1C,TRIM-MATCHING
1140,775,ZS-MGS,INACTIVE,TFE731-3D-1C,SPIRAL AIR CHARTER CC,FALCON 50,75102,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,None,3645,3645-TFE731-3D-1C,TRIM-MATCHING


In [7]:
selected_engine_trim_mismatch = engine_trim_mismatch [['ESN','REGNO', 'MTX_STATUS', 'POS_1', 'POS_2', 'POS_3', 'POS_4', 'ACMODEL', 'ACMODELID', 'ENGMODEL', 'IDENTIFIER', 'CAT']]
selected_engine_trim_mismatch.to_csv('engine_trim_mismatch.csv', index=False)
selected_engine_trim_mismatch

,ESN,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,ENGMODEL,IDENTIFIER,CAT
0,AB0033,003025,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
1,116106,003026,INACTIVE,PW119B,PW119C,None,None,DORNIER 328-100,3281,PW119B,3281-PW119B,TRIM-MISMATCHING
2,116161,003106,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
3,ED0062,5R-EJD,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
4,120304,5Y-BZI,INACTIVE,PW120A,PW121,None,None,DASH 8,6190,PW120A,6190-PW120A,TRIM-MISMATCHING
...,...,...,...,...,...,...,...,...,...,...,...,...
1137,112332,ZS-DEX,ACTIVE,TFE731-60-1C,TFE731-60-1C,TFE731-60-1C,None,900EX,6893,TFE731-60-1C,6893-TFE731-60-1C,TRIM-MATCHING
1138,101226,ZS-DFJ,ACTIVE,TFE731-5BR-1C,TFE731-5BR-1C,TFE731-5BR-1C,None,FALCON 900,6892,TFE731-5BR-1C,6892-TFE731-5BR-1C,TRIM-MATCHING
1139,115116,ZS-LAC,INACTIVE,TFE731-40-1C,TFE731-40-1C,TFE731-40-1C,None,FALCON 50EX,3647,TFE731-40-1C,3647-TFE731-40-1C,TRIM-MATCHING
1140,75102,ZS-MGS,INACTIVE,TFE731-3D-1C,TFE731-3D-1C,TFE731-3D-1C,None,FALCON 50,3645,TFE731-3D-1C,3645-TFE731-3D-1C,TRIM-MATCHING


In [13]:
selected_engine_trim_mismatch [ (selected_engine_trim_mismatch['CAT']== 'TRIM-MISMATCHING') ].reset_index(drop=True)

,ESN,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,ENGMODEL,IDENTIFIER,CAT
0,AB0033,003025,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
1,116106,003026,INACTIVE,PW119B,PW119C,None,None,DORNIER 328-100,3281,PW119B,3281-PW119B,TRIM-MISMATCHING
2,116161,003106,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
3,ED0062,5R-EJD,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
4,120304,5Y-BZI,INACTIVE,PW120A,PW121,None,None,DASH 8,6190,PW120A,6190-PW120A,TRIM-MISMATCHING
...,...,...,...,...,...,...,...,...,...,...,...,...
146,ED1524,VT-RKD,INACTIVE,PW127M,PW127F,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING
147,CF0889,VT-RSG,ACTIVE,PW308C+,PW308C,None,None,FALCON 2000EX EASY,FA2Z,PW308C+,FA2Z-PW308C+,TRIM-MISMATCHING
148,108138,XA-LEO,INACTIVE,TFE731-3CR-100S,TFE731-3C-100S,None,None,650 - CITATION III,C650,TFE731-3CR-100S,C650-TFE731-3CR-100S,TRIM-MISMATCHING
149,EB0236,ZS-XCO,INACTIVE,PW127,PW127F,None,None,ATR72,AT72,PW127,AT72-PW127,TRIM-MISMATCHING


In [5]:
A = selected_engine_trim_mismatch

In [8]:
A [A['ACMODEL']=='EMB-120ER' ]

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,ENGMODEL,IDENTIFIER,CAT
3,5H-AIH,INACTIVE,PW118,PW118B,None,None,EMB-120ER,EB12,PW118,EB12-PW118,TRIM-MISMATCHING
4,5H-KIH,INACTIVE,PW118A,PW118,None,None,EMB-120ER,EB12,PW118A,EB12-PW118A,TRIM-MISMATCHING


In [30]:
selected = A [ (A['CAT']=='TRIM-MISMATCHING') & (A['MTX_STATUS']=='ACTIVE') ]
selected_active_mismatch = list (selected['ACMODEL'].unique())
selected_active_mismatch

['FALCON 2000S',
 'FALCON 2000LX',
 'FALCON 2000EX EASY',
 'LEARJET 45',
 'FALCON 2000LXS',
 'PHENOM 100EV',
 'PRAETOR 600',
 'FALCON 50EX',
 'C-212-CD (EHAI)',
 'PHENOM 300E',
 'B429',
 'SUPER KING AIR B200',
 '650 - CITATION VI',
 'PHENOM 100',
 'G150',
 '650 - CITATION III',
 'LEARJET 35']

In [31]:
B = A [A['CAT']=='TRIM-MISMATCHING']
B

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,ENGMODEL,IDENTIFIER,CAT
0,003025,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
1,003026,INACTIVE,PW119B,PW119C,None,None,DORNIER 328-100,3281,PW119B,3281-PW119B,TRIM-MISMATCHING
2,003106,INACTIVE,PW119C,PW119B,None,None,DORNIER 328-100,3281,PW119C,3281-PW119C,TRIM-MISMATCHING
3,5H-AIH,INACTIVE,PW118,PW118B,None,None,EMB-120ER,EB12,PW118,EB12-PW118,TRIM-MISMATCHING
4,5H-KIH,INACTIVE,PW118A,PW118,None,None,EMB-120ER,EB12,PW118A,EB12-PW118A,TRIM-MISMATCHING
...,...,...,...,...,...,...,...,...,...,...,...
161,XA-LEO,INACTIVE,TFE731-3CR-100S,TFE731-3C-100S,None,None,650 - CITATION III,C650,TFE731-3CR-100S,C650-TFE731-3CR-100S,TRIM-MISMATCHING
162,XY-AJN,INACTIVE,PW127M,PW127,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING
163,YU-ALU,INACTIVE,PW127M,PW127F,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING
164,ZS-XCO,INACTIVE,PW127,PW127F,None,None,ATR72,AT72,PW127,AT72-PW127,TRIM-MISMATCHING


In [9]:
B['ACMODEL'].unique()

array(['DORNIER 328-100', 'EMB-120ER', 'ATR72', 'DASH 8', 'DASH-8-311',
       'DASH 8-106', 'DASH 8-300', 'C-12U', 'DHC-6-300',
       'ATR 72-212A (500 SERIES)', 'FALCON 2000S', 'DHC-6', 'ATR42-320',
       'DC-3', 'DASH 8-102', 'FALCON 2000LX', 'LEARJET 45', 'ATR 72-212',
       'G150', 'FALCON 2000EX EASY', 'LEARJET 35', 'LEARJET 35A',
       'LEARJET 45 (EMP)', 'ATR 42-200/-300/-320', 'ATR 72-600',
       'ATR 42-500', 'ATR 42-400/-500/-600', 'ATR 42-500 (600 VERISION)',
       'DASH 8 Q200', 'ATR42-400/-500/-600', 'BEECHCRAFT 99A',
       'FALCON 2000LXS', 'FALCON 50', 'PHENOM 100EV', 'FALCON 900B',
       'AIRLINER B99', 'PRAETOR 600', 'FALCON 50EX', 'LEGACY 450',
       'EC135(P1/P2/P3/T1/T2/T3)', 'C-212-CD (EHAI)', 'PHENOM 300E',
       'DHC-8-200', 'B429', 'SUPER KING AIR B200', 'EMB-120 BRASILIA',
       '650 - CITATION VI', 'PHENOM 100', '650 - CITATION III',
       'ATR42-300', 'DHC-6-100/200/300', 'ATR 72-212A (600 SERIES)',
       'BELL 412HP', 'DASH 8-201'], dtype=objec

In [20]:
A [A['ACMODEL']=='ATR72' ]

,REGNO,MTX_STATUS,POS_1,POS_2,POS_3,POS_4,ACMODEL,ACMODELID,ENGMODEL,IDENTIFIER,CAT
5,5R-EJD,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
20,9N-AMM,INACTIVE,PW127M,PW127E,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING
22,9S-AAD,INACTIVE,PW127M,PW127F,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING
23,9S-ABD,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
24,9S-ACE,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
25,A2-ABK,INACTIVE,PW127N,PW127M,None,None,ATR72,AT72,PW127N,AT72-PW127N,TRIM-MISMATCHING
27,B-17012,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
43,CN-COI,INACTIVE,PW127F,PW127M,None,None,ATR72,AT72,PW127F,AT72-PW127F,TRIM-MISMATCHING
54,EC-JXF,INACTIVE,PW127,PW127F,None,None,ATR72,AT72,PW127,AT72-PW127,TRIM-MISMATCHING
55,EC-KKQ,INACTIVE,PW127M,PW127F,None,None,ATR72,AT72,PW127M,AT72-PW127M,TRIM-MISMATCHING


In [ ]:
Learjet 35A

In [22]:
import pandas as pd

# Define the data
data = {
    "Aircraft": [
        "Dornier 328-100", "EMB-120ER", "ATR 72", "Dash 8", "Dash-8-311", "Dash 8-106", "Dash 8-300", "C-12U",
        "DHC-6-300", "ATR 72-212A (500 Series)", "Falcon 2000S", "DHC-6", "ATR 42-320", "DC-3", "Dash 8-102",
        "Falcon 2000LX", "Learjet 45", "ATR 72-212", "G150", "Falcon 2000EX EASy", "Learjet 35", "Learjet 35A",
        "Learjet 45 (EMP)", "ATR 42-200/-300/-320", "ATR 72-600", "ATR 42-500", "ATR 42-400/-500/-600",
        "ATR 42-500 (600 Version)", "Dash 8 Q200", "ATR42-400/-500/-600", "Beechcraft 99A", "Falcon 2000LXS",
        "Falcon 50", "Phenom 100EV", "Falcon 900B", "Airliner B99", "Praetor 600", "Falcon 50EX", "Legacy 450",
        "EC135 (P1/P2/P3/T1/T2/T3)", "C-212-CD (EHAI)", "Phenom 300E", "DHC-8-200", "Bell 429", "Super King Air B200",
        "EMB-120 Brasilia", "650 - Citation VI", "Phenom 100", "650 - Citation III", "ATR 42-300", "DHC-6-100/200/300",
        "ATR 72-212A (600 Series)", "Bell 412HP", "Dash 8-201"
    ],
    "Engine Trim (Variant)": [
        "Pratt & Whitney PW119B", "Pratt & Whitney Canada PW118A", "Pratt & Whitney Canada PW127", "Various (typically Pratt & Whitney PW120 series)",
        "Pratt & Whitney Canada PW123", "Pratt & Whitney Canada PW121", "Pratt & Whitney Canada PW123", "Pratt & Whitney Canada PT6A-42",
        "Pratt & Whitney Canada PT6A-27", "Pratt & Whitney Canada PW127F", "Pratt & Whitney Canada PW308C", "Pratt & Whitney Canada PT6A (varies with series: -20, -27, etc.)",
        "Pratt & Whitney Canada PW121", "Originally Wright R-1820 Cyclone 9; many retrofitted with Pratt & Whitney R-1830 Twin Wasp",
        "Pratt & Whitney Canada PW120A", "Pratt & Whitney Canada PW308C", "Honeywell TFE731-20AR", "Pratt & Whitney Canada PW127",
        "Honeywell TFE731-40AR", "Pratt & Whitney Canada PW308C", "Garrett TFE731-2-2B", "Garrett TFE731-2-2B", "Honeywell TFE731-20AR",
        "Pratt & Whitney Canada PW120, PW121", "Pratt & Whitney Canada PW127M", "Pratt & Whitney Canada PW127E", "Pratt & Whitney Canada PW127 series",
        "Pratt & Whitney Canada PW127M", "Pratt & Whitney Canada PW123D", "Pratt & Whitney Canada PW127 series", "Pratt & Whitney Canada PT6A-27",
        "Pratt & Whitney Canada PW308C", "Garrett TFE731-3-1C", "Pratt & Whitney Canada PW617F1-E", "Garrett TFE731-5BR-1C",
        "Pratt & Whitney Canada PT6A-27", "Honeywell HTF7500E", "Garrett TFE731-40", "Honeywell HTF7500E",
        "Turbomeca Arrius 2B2, Pratt & Whitney Canada PW206B2/B3", "Garrett TPE331-10R-513C", "Pratt & Whitney Canada PW535E1",
        "Pratt & Whitney Canada PW123D", "Pratt & Whitney Canada PW207D1", "Pratt & Whitney Canada PT6A-42", "Pratt & Whitney Canada PW118",
        "Honeywell TFE731-3B-100S", "Pratt & Whitney Canada PW617F-E", "Honeywell TFE731-3B-100S", "Pratt & Whitney Canada PW120",
        "Pratt & Whitney Canada PT6A (varies with series: -20, -27, etc.)", "Pratt & Whitney Canada PW127M", "Pratt & Whitney Canada PT6T-3B",
        "Pratt & Whitney Canada PW123D"
    ],

    "Sources": [
    "https://en.wikipedia.org/wiki/Dornier_328", 
    "https://en.wikipedia.org/wiki/Embraer_EMB_120_Brasilia", 
    "https://en.wikipedia.org/wiki/ATR_72", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/Beechcraft_C-12_Huron", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_DHC-6_Twin_Otter", 
    "https://en.wikipedia.org/wiki/ATR_72", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_2000", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_DHC-6_Twin_Otter", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/Douglas_DC-3", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_2000", 
    "https://en.wikipedia.org/wiki/Learjet_45", 
    "https://en.wikipedia.org/wiki/ATR_72", 
    "https://en.wikipedia.org/wiki/Gulfstream_G150", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_2000", 
    "https://en.wikipedia.org/wiki/Learjet_35", 
    "https://en.wikipedia.org/wiki/Learjet_35", 
    "https://en.wikipedia.org/wiki/Learjet_45", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/ATR_72", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/Beechcraft_Model_99", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_2000", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_50", 
    "https://en.wikipedia.org/wiki/Embraer_Phenom_100", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_900", 
    "https://en.wikipedia.org/wiki/Beechcraft_Model_99", 
    "https://en.wikipedia.org/wiki/Embraer_Praetor_600", 
    "https://en.wikipedia.org/wiki/Dassault_Falcon_50", 
    "https://en.wikipedia.org/wiki/Embraer_Legacy_450", 
    "https://en.wikipedia.org/wiki/Eurocopter_EC135", 
    "https://en.wikipedia.org/wiki/CASA_C-212_Aviocar", 
    "https://en.wikipedia.org/wiki/Embraer_Phenom_300", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8", 
    "https://en.wikipedia.org/wiki/Bell_429", 
    "https://en.wikipedia.org/wiki/Beechcraft_Super_King_Air", 
    "https://en.wikipedia.org/wiki/Embraer_EMB_120_Brasilia", 
    "https://en.wikipedia.org/wiki/Cessna_Citation_III", 
    "https://en.wikipedia.org/wiki/Embraer_Phenom_100", 
    "https://en.wikipedia.org/wiki/Cessna_Citation_III", 
    "https://en.wikipedia.org/wiki/ATR_42", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_DHC-6_Twin_Otter", 
    "https://en.wikipedia.org/wiki/ATR_72", 
    "https://en.wikipedia.org/wiki/Bell_412", 
    "https://en.wikipedia.org/wiki/De_Havilland_Canada_Dash_8"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_path = "aircraft_engine_trim_variants.csv"
df.to_csv(csv_path, index=False)

csv_path


'aircraft_engine_trim_variants.csv'

In [34]:
df

,Aircraft,Engine Trim (Variant),Sources
0,Dornier 328-100,Pratt & Whitney PW119B,https://en.wikipedia.org/wiki/Dornier_328
1,EMB-120ER,Pratt & Whitney Canada PW118A,https://en.wikipedia.org/wiki/Embraer_EMB_120_...
2,ATR 72,Pratt & Whitney Canada PW127,https://en.wikipedia.org/wiki/ATR_72
3,Dash 8,Various (typically Pratt & Whitney PW120 series),https://en.wikipedia.org/wiki/De_Havilland_Can...
4,Dash-8-311,Pratt & Whitney Canada PW123,https://en.wikipedia.org/wiki/De_Havilland_Can...
5,Dash 8-106,Pratt & Whitney Canada PW121,https://en.wikipedia.org/wiki/De_Havilland_Can...
6,Dash 8-300,Pratt & Whitney Canada PW123,https://en.wikipedia.org/wiki/De_Havilland_Can...
7,C-12U,Pratt & Whitney Canada PT6A-42,https://en.wikipedia.org/wiki/Beechcraft_C-12_...
8,DHC-6-300,Pratt & Whitney Canada PT6A-27,https://en.wikipedia.org/wiki/De_Havilland_Can...
9,ATR 72-212A (500 Series),Pratt & Whitney Canada PW127F,https://en.wikipedia.org/wiki/ATR_72


In [ ]:
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = 'NHVM134'
DATABASE_NAME = 'FINANCE'

connection_string = f"""
    DRIVER={{{DRIVER_NAME}}};
    SERVER={SERVER_NAME};
    DATABASE={DATABASE_NAME};
    Trust_Connection=yes;
"""

conn = pyodbc.connect(connection_string)
print(conn)

In [ ]:
customer_data = pd.read_sql("SELECT * FROM [AcConfig] ", conn) 
customer_data

In [4]:
customer_data.columns

Index(['IS_ACTIVE_CUSTOMER', 'CUSTOMER_NAME', 'PRIMARYENTERPRISEID',
       'AC_OEM_NAME', 'AC_MODEL_NAME', 'AC_PROFILE_ID', 'AC_ASSET_ID', 'REGNO',
       'AC_SN', 'AC_STATUS', 'AC_ECTM_SUBSCRIPTION', 'NBR_ENGINE', 'ENG_POS',
       'ENG_POS_TEXT', 'ENG_INSTALLED_DATE', 'ENG_OEM_NAME', 'ENG_MODEL_MTX',
       'ENG_MODEL_ALIAS_MTX', 'ENG_MODEL_TRIM', 'ENG_PROFILE_ID',
       'ENG_ASSET_ID', 'ORIGINAL_ENG_SN', 'NOZZLE_TYPE', 'N1_COMPENSATOR',
       'ENG_SN', 'ENG_STATUS', 'IS_DELETED', 'ENG_ECTM_SUBSCRIPTION',
       'BASELINE_DATE', 'ENG_LOANER', 'PRODUCTION_PROFILE_FLAG',
       'ENG_MODEL_TRIM_ISSUE', 'ENG_N1_COMPENSATOR_ISSUE', 'NOZZLE_TYPE_ISSUE',
       'ENG_ASSET_ID_ISSUE', 'AC_ASSET_ID_ISSUE', 'SUBSCRIPTION_ISSUE',
       'EHM_ONLY', 'Wave', 'IS_NEW', 'RunDt', 'ProgramType', 'CONTRACT_NUMBER',
       'CONTRACT_STATUS', 'CONTRACT_START_DATE', 'CONTRACT_EXPIRATION_DATE',
       'DTU_ID', 'INSTALL_ID', 'EDU_ID', 'FAST_ID', 'AC_MANAGED_BY', 'SFIM_ID',
       'AC_HRS_DATE', 'AC_HRS_

In [24]:
B = customer_data[[ 'REGNO', 'IS_ACTIVE_CUSTOMER', 'ENG_POS', 'ENG_POS_TEXT',  'ENG_OEM_NAME', 'ENG_MODEL_MTX', 'ENG_MODEL_TRIM' , 'AC_MODEL_NAME']]
B ['IS_ACTIVE_CUSTOMER'] = B['IS_ACTIVE_CUSTOMER'].replace ({'1': 'INACTIVE', '0': 'ACTIVE'})
B

C:\Users\bezyb\AppData\Local\Temp\ipykernel_3996\791025012.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B ['IS_ACTIVE_CUSTOMER'] = B['IS_ACTIVE_CUSTOMER'].replace ({'1': 'INACTIVE', '0': 'ACTIVE'})


,REGNO,IS_ACTIVE_CUSTOMER,ENG_POS,ENG_POS_TEXT,ENG_OEM_NAME,ENG_MODEL_MTX,ENG_MODEL_TRIM,AC_MODEL_NAME
0,N899B,INACTIVE,1,side,PWC,PW530A,PW530A,550 - BRAVO
1,N318GL,INACTIVE,1,side,PWC,PW530A,PW530A,550 - BRAVO
2,N971KA,INACTIVE,2,side,PWC,PT6A-60A/-60AG/-61,PT6A-60A,SUPER KING AIR B300 (350I)
3,N69LT,INACTIVE,1,side,PWC,PT6A-67B/-67P,PT6A-67P,PC-12/47E
4,N503CB,INACTIVE,1,side,PWC,PT6A-21/27/28,PT6A-21,KING AIR 90
...,...,...,...,...,...,...,...,...
111483,C-GPCY,INACTIVE,2,side,PWC,PT6A-65B/65R/65AR/65AG,PT6A-65B,BEECH 1900D
111484,N328LM,INACTIVE,1,side,GENERAL ELECTRIC,GE PASSPORT 20-19BB1A,GE PASSPORT 20-19BB1AG03,BD700-2A12 (GLOBAL 7500)
111485,N328LM,INACTIVE,2,side,GENERAL ELECTRIC,GE PASSPORT 20-19BB1A,GE PASSPORT 20-19BB1AG04,BD700-2A12 (GLOBAL 7500)
111486,9H-DNAGEZ1,INACTIVE,1,side,HONEYWELL,AS907-2-1A,AS907-2-1A,CH350


In [25]:
B['ENG_POS_TEXT'].unique()

array(['side', 'center'], dtype=object)

In [26]:
B [ B['ENG_POS_TEXT']=='center' ]

,REGNO,IS_ACTIVE_CUSTOMER,ENG_POS,ENG_POS_TEXT,ENG_OEM_NAME,ENG_MODEL_MTX,ENG_MODEL_TRIM,AC_MODEL_NAME
82,N81SN,INACTIVE,2,center,HONEYWELL,TFE731-20/-40/-50/-60,TFE731-60-1C,900EX
83,N900TE,INACTIVE,2,center,HONEYWELL,TFE731-5BR,TFE731-5BR-1C,FALCON 900
117,N115KG,INACTIVE,2,center,HONEYWELL,TFE731-20/-40/-50/-60,TFE731-60-1C,FALCON 900LX
230,C-FRJY,INACTIVE,2,center,HONEYWELL,LF507-1F,LF507-1F,146-RJ100
285,PH-GWS,INACTIVE,2,center,PWC,PW307A,PW307A,FALCON 7X
...,...,...,...,...,...,...,...,...
111229,N4VFEZ1,INACTIVE,2,center,PWC,PW307A,PW307A,FALCON 7X
111313,N206FJAMP,INACTIVE,2,center,HONEYWELL,TFE731-20/-40/-50/-60,TFE731-60-1C,FALCON 900 EX EASY
111337,N8XDEMO,INACTIVE,2,center,PWC,PW307D,PW307D,FALCON 8X
111381,N40LBEZ1,INACTIVE,2,center,HONEYWELL,TFE731-20/-40/-50/-60,TFE731-60-1C,FALCON 900LX


In [27]:
B [ B['REGNO']=='2-NYAW' ]

,REGNO,IS_ACTIVE_CUSTOMER,ENG_POS,ENG_POS_TEXT,ENG_OEM_NAME,ENG_MODEL_MTX,ENG_MODEL_TRIM,AC_MODEL_NAME
48474,2-NYAW,INACTIVE,2,center,HONEYWELL,TFE731-3A-200G,TFE731-3D-1C,FALCON 50
54090,2-NYAW,INACTIVE,3,side,HONEYWELL,TFE731-3A-200G,TFE731-3D-1C,FALCON 50
57198,2-NYAW,INACTIVE,1,side,HONEYWELL,TFE731-3A-200G,TFE731-3D-1C,FALCON 50


In [15]:
inactive_customers= customer_data [(customer_data['IS_ACTIVE_CUSTOMER'] == '0') & (customer_data['REGNO'].notna())].reset_index(drop=True)
inactive_customers.to_csv('inactive_customers.csv', index = False)
inactive_customers

,IS_ACTIVE_CUSTOMER,CUSTOMER_NAME,PRIMARYENTERPRISEID,AC_OEM_NAME,AC_MODEL_NAME,AC_PROFILE_ID,AC_ASSET_ID,REGNO,AC_SN,AC_STATUS,...,IS_SNOOZE,DDA_START_DATE,DDA_START_TYPE,RECORDER_TYPE,TRANSMITTER_TYPE,LAST_RETREND_DATE,LATEST_RETREND_STATUS,LAST_ALGO_VERSION,ALGO_VERSION_CNT,LATEST_RETREND_STATUS_FILE
0,0,None,None,TEXTRON AVIATION INC.,HAWKER 800XP,911722.DB1,170624.0,N110GD-AVT,258713-AVT,1,...,None,None,None,None,None,NaT,None,None,NaN,None
1,0,None,None,TEXTRON AVIATION INC.,HS-125-700A,96796.DB1,1627.0,N141AE,257152/NA0306,1,...,None,None,None,None,None,2017-05-21 11:01:08,Completed,None,NaN,None
2,0,HAWKER BEECHCRAFT SERVICES - FL,14150.DB1,TEXTRON AVIATION INC.,KING AIR C90A,448905.DB1,74035.0,N111KC,LJ-1258,1,...,None,None,None,None,None,NaT,None,None,NaN,None
3,0,None,None,PILATUS AIRCRAFT LTD,PC-12/47E,870037.DB1,169128.0,MSN1382,1382,1,...,None,None,None,None,None,2019-02-24 03:27:24,Completed,None,NaN,None
4,0,None,None,PILATUS AIRCRAFT LTD,PC-12/47E,821764.DB1,166964.0,M-DAVE,1301,1,...,None,None,None,None,None,2019-02-24 03:26:53,Completed,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,0,None,None,GULFSTREAM AEROSPACE CORPORATION,GVII-G600,1399406.DB1,346698.0,N647GD,73147,0,...,None,None,None,None,None,NaT,None,None,NaN,None
1862,0,None,None,EMBRAER,PHENOM 300,1401304.DB1,346764.0,N-50500792,50500792,1,...,None,None,None,None,None,NaT,None,None,NaN,None
1863,0,None,None,EMBRAER,PHENOM 300,1401304.DB1,346764.0,N-50500792,50500792,1,...,None,None,None,None,None,NaT,None,None,NaN,None
1864,0,None,None,GULFSTREAM AEROSPACE CORPORATION,G650ER,1383920.DB1,345769.0,N668GS,6568,0,...,None,None,None,None,None,NaT,None,None,NaN,None


# from Oracle SQL